# Ingest GitHub Issues into Qdrant

In [1]:
#r "nuget: Azure.AI.OpenAI, 1.0.0-beta.9"
#r "nuget: Qdrant.Client, 1.6.0-alpha.1"

Installed Packages Azure.AI.OpenAI, 1.0.0-beta.9 Qdrant.Client, 1.6.0-alpha.1

In [2]:
#r "nuget: Octokit, 9.0.0"
#r "nuget: Octokit.Reactive, 9.0.0"

Installed Packages Octokit, 9.0.0 Octokit.Reactive, 9.0.0

In [3]:
#r "nuget:Microsoft.DotNet.Interactive.AIUtilities, 1.0.0-beta.23557.4"

Installed Packages Microsoft.DotNet.Interactive.AIUtilities, 1.0.0-beta.23557.4

Loading extension script from `/home/codespace/.nuget/packages/microsoft.dotnet.interactive.aiutilities/1.0.0-beta.23557.4/interactive-extensions/dotnet/extension.dib`

In [4]:
using Azure;
using Azure.AI.OpenAI;
using Microsoft.DotNet.Interactive;
using Microsoft.DotNet.Interactive.AIUtilities;
using Octokit;

In [5]:
var azureOpenAIKey = await Kernel.GetPasswordAsync("Provide your OPEN_AI_KEY");
var azureOpenAIEndpoint = await Kernel.GetInputAsync("Provide the OPEN_AI_ENDPOINT");
var embeddingDeployment = await Kernel.GetInputAsync("Provide embedding name");

## Access to GitHub
You will need access token with rights to query and update issues.

In [ ]:
var githubKey = await Kernel.GetPasswordAsync("Provide your Github api key");
var repoName = await Kernel.GetInputAsync("Provide repo");
var org = await Kernel.GetInputAsync("Provide org");

In [6]:
OpenAIClient openAIClient = new (new System.Uri(azureOpenAIEndpoint), new AzureKeyCredential(azureOpenAIKey.GetClearTextPassword()));

In [ ]:
var options = new ApiOptions();
var gitHubClient = new GitHubClient(new ProductHeaderValue("notebook"));

if (!string.IsNullOrEmpty(githubKey.GetClearTextPassword())) {
    Console.WriteLine("Using GitHub API token");
    var tokenAuth = new Credentials(githubKey.GetClearTextPassword());
    gitHubClient.Credentials = tokenAuth;
} else {
    Console.WriteLine("Using anonymous GitHub API");
}

The code below is using the Octokit library, which is a .NET client for interacting with the GitHub API.

The first part of the code is creating a new instance of `RepositoryIssueRequest` named `last6Months`. This object is used to specify the parameters for a request to fetch issues from a GitHub repository. In this case, the `Filter` property is set to `IssueFilter.All`, which means that the request will return all issues regardless of their state (open, closed, etc.). The `Since` property is set to a date that is six months prior to the current date (`DateTimeOffset.UtcNow.Subtract(TimeSpan.FromDays(30*6))`). This means that the request will return only the issues that have been updated in the last six months.

The second part of the code is making an asynchronous request to fetch all issues for a specific repository. The `GetAllForRepository` method of the `Issue` class in the `gitHubClient` object is used to make this request. The `org` and `repoName` variables are used to specify the organization and the name of the repository from which to fetch the issues. The method returns a list of all issues in the specified repository. The `await` keyword is used to wait for the method to complete execution before moving on to the next line of code. This is necessary because the method is asynchronous, meaning it runs in the background and may not complete immediately.

In [ ]:
var last6Months = new RepositoryIssueRequest
{
    Filter = IssueFilter.All,
    Since = DateTimeOffset.UtcNow.Subtract(TimeSpan.FromDays(30*6))
};
var allIssues = await gitHubClient.Issue.GetAllForRepository(org, repoName);

In [7]:
public record GitHubIssue(string Title, string Text, string Url);
public record TextWithEmbedding(string Text, float[] Embedding);
public class IssueWithChunks
{
    public GitHubIssue Issue {get;set;}
    public List<TextWithEmbedding> Chunks {get;set;}
}

In [ ]:
var dataCollection = 
    allIssues
        .Select(issue => new IssueWithChunks{
            Issue=new GitHubIssue(issue.Title,issue.Body,issue.HtmlUrl), 
            Chunks= new ()
        })
        .ToArray();

## Helper functions to save and load to disk

In [8]:
using System.IO;
using System.Text.Json;
using System.Text.Json.Serialization;

public async Task SaveDataToFileAsync(IEnumerable<IssueWithChunks> data, string fileName)
{ 
    var filePath = Path.Combine("..","..","..","Data",fileName);
    var issuesJson = JsonSerializer.Serialize(data,new JsonSerializerOptions(JsonSerializerOptions.Default){WriteIndented=true});
    await File.WriteAllTextAsync(filePath, issuesJson);
}

public async Task<IssueWithChunks[]> LoadDataFromFileAsync(string fileName)
{
    var filePath = Path.Combine("..","..","..","Data",fileName);
    var text = await File.ReadAllTextAsync(filePath);
    return JsonSerializer.Deserialize<IssueWithChunks[]>(text);
}

In [8]:
await SaveDataToFileAsync(dataCollection, "issues.json");

Error: (1,22): error CS0103: The name 'dataCollection' does not exist in the current context

## Chunk issues

In [12]:
var dataCollection = await LoadDataFromFileAsync("issues.json");

In [13]:
var tokenizer = await Tokenizer.CreateAsync(TokenizerModel.ada2);

foreach(var item in dataCollection)
{
    var fullText = item.Issue.Text;
    if(string.IsNullOrWhiteSpace(fullText))
        continue;

    var chunks = 
        tokenizer
            .ChunkByTokenCountWithOverlap(fullText, 3000, 50)
            .Select(t => 
$"""
Issue Title: {item.Issue.Title}
Issue Url: {item.Issue.Url} 
Chunk: 
{t}
""")
            .Chunk(16)
            .ToArray();

    foreach(var chunk in chunks)
    {
        
        var embeddingResponse = await openAIClient.GetEmbeddingsAsync(new EmbeddingsOptions(embeddingDeployment,chunk));
        item.Chunks.AddRange(
            embeddingResponse.Value.Data.Select(d => 
                new TextWithEmbedding(chunk[d.Index],d.Embedding.ToArray())));
    }
}

In [2]:
await SaveDataToFileAsync(dataCollection, "issueWithEmbeddings.json");

Error: (1,7): error CS0103: The name 'SaveDataToFileAsync' does not exist in the current context
(1,27): error CS0103: The name 'dataCollection' does not exist in the current context

In [ ]:
dataCollection.Take(5).DisplayTable();

In [ ]:
await SaveDataToFileAsync(dataCollection, "issueWithEmbeddings.json");

## Naive Search

In [15]:
var dataCollection = await LoadDataFromFileAsync("issueWithEmbeddings.json");

In [16]:
public string[] NaiveSearch(string query, IEnumerable<IssueWithChunks> data,int resultLimit = 1)
{
    return data.Where(d => d.Issue.Text?.Contains(query)==true).Select(d => d.Issue.Text).Take(resultLimit).ToArray();
}

In [17]:
NaiveSearch("What are the latest issues for AOT",dataCollection).Display();

[ ]

## Embedding Search

In [18]:
public async Task<string[]> EmbeddingSearchAsync(string query, IEnumerable<IssueWithChunks> data,int resultLimit = 1)
{
    var embeddingResponse = await openAIClient.GetEmbeddingsAsync(new EmbeddingsOptions(embeddingDeployment,new [] {query}));
    var embeddingVector = embeddingResponse.Value.Data[0].Embedding.ToArray();

    var searchResults = 
        data
        .SelectMany(d => d.Chunks)
        .ScoreBySimilarityTo(embeddingVector,new CosineSimilarityComparer<float[]>(f => f),c => c.Embedding)
        .OrderByDescending(e => e.Value)
        .Where(e => e.Value > 0.5)
        .Take(resultLimit)
        .Select(e => e.Key.Text)
        .ToArray();
    
    return searchResults;
}

Error: (3,35): error CS0103: The name 'openAIClient' does not exist in the current context
(3,89): error CS0103: The name 'embeddingDeployment' does not exist in the current context

In [ ]:
(await EmbeddingSearchAsync("What are the latest issues for AOT", dataCollection, 3)).Display();

## Store in DB

In [20]:
docker run -d -p 6333:6333 -p 6334:6334 -v "$pwd/qdrant_storage:/qdrant/storage:z" qdrant/qdrant

9d4850aa632fa14361253b64160a6f43b7767794f37b9f6afdf3e61f275961aa


In [21]:
using Qdrant.Client;
using Qdrant.Client.Grpc;

In [22]:
var client = new QdrantClient(host: "localhost",port: 6334,https:false);

In [30]:
var collectionName = "gh_issues";

In [26]:
await client.CreateCollectionAsync(collectionName, new VectorParams { Size=1536, Distance=Distance.Cosine})

In [25]:
var vectors = 
    dataCollection
        .Where(d => d.Chunks.Count > 0)
        .SelectMany(d => d.Chunks)
        .ToList();
  

In [32]:
var points = vectors.Select(vector => 
{
    var point = new PointStruct
    {
        Id = new PointId { Uuid = Guid.NewGuid().ToString() },
        Vectors = vector.Embedding,
        Payload = 
            {
                ["text"] = vector.Text
            }
    };
    return point;
}).ToList();


In [33]:
await client.UpsertAsync(collectionName,points);

## Get Count

In [35]:
await client.CountAsync(collectionName)

5

## Search with Qdrant

In [47]:
public async Task<string[]> SearchWithQdrantAsync(string query, string collectionName, int resultLimit = 1)
{
    var embeddingResponse = await openAIClient.GetEmbeddingsAsync(new EmbeddingsOptions(embeddingDeployment,new [] {query}));
    var embeddingVector = embeddingResponse.Value.Data[0].Embedding.ToArray();

    var results = await client.SearchAsync(collectionName,embeddingVector, limit:(ulong)resultLimit);
    return results.Select(r => r.Payload["text"].StringValue).ToArray();
}

In [48]:
(await SearchWithQdrantAsync("What are the latest issues for AOT", collectionName, 3)).Display();

[ Issue Title: ConfigurationBinder emits AOT warnings even when using Source Generator when passing result into a method
Issue Url: https://github.com/dotnet/runtime/issues/94544 
Chunk: 
### Description

When passing the result of the ConfigurationBinder `.Get<T>()` directly into a method, I'm still getting AOT/trim warnings.

### Reproduction Steps

Build the following app:

```xml
<Project Sdk="Microsoft.NET.Sdk">

 <PropertyGroup>
 <OutputType>Exe</OutputType>
 <TargetFramework>net8.0</TargetFramework>
 <ImplicitUsings>enable</ImplicitUsings>
 <Nullable>enable</Nullable>
 <EnableConfigurationBindingGenerator>true</EnableConfigurationBindingGenerator>
 <EmitCompilerGeneratedFiles>true</EmitCompilerGeneratedFiles>
 <PublishAot>true</PublishAot>
 </PropertyGroup>

 <ItemGroup>
 <PackageReference Include="Microsoft.Extensions.Configuration.Binder" Version="9.0.0-alpha.1.23558.3" /> 
 <PackageReference Include="Microsoft.Extensions.Configuration" Version="9.0.0-alpha.1.23558.3" />
 </ItemGroup>

</Project>
```

```C#
using Microsoft.Extensions.Configuration;

IConfigurationSection c = new ConfigurationBuilder().Build().GetSection("Options");
#if WORKS
var o = c.Get<MyOptions>();
Some.Method(o);
#else
Some.Method(c.Get<MyOptions>());
#endif

internal static class Some
{
 public static void Method(MyOptions? options) { }
}

public class MyOptions
{
 public int MaxRetries { get; set; }
}
```

### Expected behavior

I shouldn't get AOT/trim warnings because I'm using the Source Generator.

### Actual behavior

```
1>C:\Users\eerhardt\source\repos\ConsoleApp101\ConsoleApp101\Program.cs(4,13,4,31): warning IL3050: Using member 'Microsoft.Extensions.Configuration.ConfigurationBinder.Get<T>(IConfiguration)' which has 'RequiresDynamicCodeAttribute' can break functionality when AOT compiling. Binding strongly typed objects to configuration values requires generating dynamic code at runtime, for example instantiating generic types.
1>C:\Users\eerhardt\source\repos\ConsoleApp101\ConsoleApp101\Program.cs(4,13,4,31): warning IL2026: Using member 'Microsoft.Extensions.Configuration.ConfigurationBinder.Get<T>(IConfiguration)' which has 'RequiresUnreferencedCodeAttribute' can break functionality when trimming application code. In case the type is non-primitive, the trimmer cannot statically analyze the object's type so its members may be trimmed.
```

### Regression?

_No response_

### Known Workarounds

Capture the object into a variable on a separate line, and pass that into the method:

```C#
var o = c.Get<MyOptions>();
Some.Method(o);
```

### Configuration

_No response_

### Other information

_No response_, Issue Title: Ensure that a native code version only ever gets one entry point that doesn't change
Issue Url: https://github.com/dotnet/runtime/pull/94542 
Chunk: 
- Fixed `MethodDesc::SetNativeCodeInterlocked` to use the passed-in expected entry point instead of the current entry point. It seems this is how it used to work before https://github.com/dotnet/runtime/pull/57707.
- It's possible for a method to get jitted multiple times on the same thread due to reentry through class construction. Once a native code version is updated with an entry point, it can run and there are times when it's necessary to look up the native code version corresponding to the currently running code, such as when installing patchpoints. That would be impossible if the entry point is changed meanwhile to a different entry point.

Fixes https://github.com/dotnet/runtime/issues/93849, Issue Title: Reflection fails when working with struct implementing interface that implements inherited static abstract members
Issue Url: https://github.com/dotnet/runtime/issues/94543 
Chunk: 
### Description

Some reflection API calls cause program to fail when:
- struct _A_ impements interface _B_
- interface _B_ inherits interface _C_ and provides implementation of static abstract members